In [ ]:
import numpy as np
import quaternion
from mpl_toolkits.mplot3d import axes3d
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display


In [ ]:
# Define celestial coordinates: hours of right ascension: like longitude, parallels of declination: like lattitude, epoch: reference time for time-varying quantities
# We want to track relative motion of a general object, so we won't worry about epoch here
NUM_PARALLELS = 5 #by default, show 5 lines above and below celestial equator

# Construct celestial wireframe - unit ball with radius 1
# Earth is 13 deg off axis - ecliptic hours are based on the sun's motion, so this system isn't 100% correct

theta, phi = np.linspace(0, 2 * np.pi, 24), np.linspace(0, np.pi, 2*NUM_PARALLELS)
THETA, PHI = np.meshgrid(theta, phi)
R = 1.0
X = R * np.sin(PHI) * np.cos(THETA)
Y = R * np.sin(PHI) * np.sin(THETA)
Z = R * np.cos(PHI)

In [ ]:
def normalize(vec):
    '''
    Utility method to normalize vectors
    '''
    norm = np.linalg.norm(vec)
    return np.array([comp/norm for comp in vec])

def transform(pt):
    '''
    Transforms a point from Earth to celestial coordinates by rotating 23.5deg downward (xy positioning is assumed to be correct)
    pt - A unit vector with x,y,z coordinates
    Returns a unit vector in celestial coordinates
    '''
    a=np.deg2rad(23.5)
    r = make_rot_quat(a, 0, 1, 0)
    p = np.quaternion(0, *pt)
    return quaternion.as_vector_part(r*p*r.conjugate())

def make_rot_quat(a,x,y,z):
    '''
    Makes a quaternion that represents rotation around a vector given angle and the vector
    '''
    return np.quaternion(np.cos(a/2), np.sin(a/2)*x, np.sin(a/2)*y, np.sin(a/2)*z)


In [ ]:
# x,y,z coordinates of North Star - pretend Polaris is exactly on the celestial pole
polaris = transform([0,0,1])

In [ ]:
# Plot the motion of an object around the north star in the celestial spheres

def plot_star(star, NF=100, **kwargs):
    '''
    Plots a star moving over 24 hours
    :param star: 3-D coordinates of the star to visualize
    :param int NF: number of frames to plot
    '''
    # Clear the axes and plot base wireframe
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    wframe = ax.plot_wireframe(X, Y, Z, rstride=1, cstride=1, color='k', linewidth=0.5)
    pole = ax.plot(*[[coord] for coord in polaris], 'bo')

    # Create star in the sky and plot initial pos
    _star, = ax.plot(*[[coord] for coord in polaris], 'o')

    # Setting the axes properties
    ax.set_box_aspect([1,1,1])

    ax.set_xlim3d([-1.0, 1.0])
    ax.set_xlabel('X')

    ax.set_ylim3d([-1.0, 1.0])
    ax.set_ylabel('Y')

    ax.set_zlim3d([-1.0, 1.0])
    ax.set_zlabel('Z')

    def update(i):
        '''
        Animate motion of the star 
        '''
        r = make_rot_quat(2*np.pi*i/NF, *transform((0,0,1)))
        new = quaternion.as_vector_part(r*np.quaternion(0, *star)*r.conjugate()) 
        assert np.linalg.norm(new) > 0.9, "it's not a unit vec WTC"
        _star.set_data(*[[coord] for coord in new][:2])
        _star.set_3d_properties([new[2]])

        return star,

    anim = FuncAnimation(fig, update, fargs=kwargs,  frames=NF, interval=50, blit=False)
    display(HTML(anim.to_jshtml()))
    plt.close()


In [ ]:
# Visualize the motion of a random star over 24h
plot_star(normalize((1,1,1)))

In [ ]:
def find_star(ct, sp):
    '''
    Find move the camera to center on a star given a (possibly) misaligned axis
    Returns declination (up and down) and rotation the equatorial tracker would need to make
    '''
    # Assume that camera will be looking 'up' in general, assume declination can always be positive
    dec = np.arccos(np.dot(ct, sp))

    # Find angle relative to straight above north star
    up = np.array([0,0,1])
    uperp = normalize(up-np.dot(polaris, up)*polaris)

    rot = np.arccos(np.dot(normalize(sp-polaris), uperp))

    return dec, rot


def calc_dev(t, ct, sp):
    '''
    Calculate deviation as a function of time.
    Plots the declination angle of the tracker at time step t.
    :param t: Time value from 0 to 100.
    '''
    # Calc current pos of the star
    r = make_rot_quat(np.pi*2*t/100, *transform((0,0,1)))
    q = np.quaternion(0, *sp)
    s_curr = quaternion.as_vector_part(r*q*r.conjugate())
    assert 0.98 < np.linalg.norm(ct) < 1.01
    assert 0.98 < np.linalg.norm(s_curr) < 1.01

    return np.arccos(np.dot(ct, s_curr))
    
    

In [ ]:
def plot_tracker(star, tracker, NF=100):
    '''
    Plots the motion of a star and records the two motions the tracker would need to make.
    '''
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    wframe = ax.plot_wireframe(X, Y, Z, rstride=1, cstride=1, color='k', linewidth=0.5)
    pole = ax.plot(*[[coord] for coord in polaris], 'bo')

    # Create star in the sky and plot initial pos
    _star, = ax.plot(*[[coord] for coord in star_pos], 'ro')
    _tracker, = ax.plot(*[[coord] for coord in star])
    plot = [_star, _tracker]


    # Setting the axes properties
    ax.set_box_aspect([1,1,1])

    ax.set_xlim3d([-1.0, 1.0])
    ax.set_xlabel('X')

    ax.set_ylim3d([-1.0, 1.0])
    ax.set_ylabel('Y')

    ax.set_zlim3d([-1.0, 1.0])
    ax.set_zlabel('Z')

    # Plot tracker axis of rotation
    coords = np.vstack(([0,0,0], tracker))
    ax.plot(*np.transpose(coords))

    def update(i):
        '''
        Animate motion of the star and tracker 
        '''
        r = make_rot_quat(np.pi*2*i/NF, *transform([0,0,1]))
        q = np.quaternion(0, *star)
        new = quaternion.as_vector_part(r*q*r.conjugate()) 
        coords = np.vstack(([0,0,0], new, tracker))
        _tracker.set_data(*np.transpose(coords)[:2])
        _tracker.set_3d_properties(np.transpose(coords)[2])
        _star.set_data(*[[i] for i in new[:2]])
        _star.set_3d_properties([new[2]])

        return plot

    anim = FuncAnimation(fig, update, frames=NF, interval=50, blit=False)
    display(HTML(anim.to_jshtml()))
    plt.close()

In [ ]:
# Simulate a tracker (not necessarily aligned at north pole) and a simulated star and plot the change over time
ascension = 60
tracker =  normalize((np.cos(np.deg2rad(ascension)),0, np.sin(np.deg2rad(ascension))))
star_pos = normalize([2, 4, 3])


s0 = calc_dev(0, polaris, star_pos)
t = np.arange(0, 200)

dev = [np.rad2deg(calc_dev(i, tracker, star_pos)-s0) for i in t]
sd = [calc_dev(i, polaris, star_pos)-s0 for i in t]

cam, star = normalize([0, 1, 1]), normalize([2, 4, 3])
find_star(cam, star)
dev = [calc_dev(i, cam, star)-s0 for i in t]

plot_tracker(star_pos, tracker)

plt.title("Deviation angle over time")
plt.xlabel("Time steps")
plt.ylabel("Deviation in degrees")
plt.plot(t, dev)
plt.plot(t, sd)
